In [1]:
import pandas as pd
import os
from collections import defaultdict
from tqdm import tqdm
import cv2
import numpy as np
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import imgaug.augmenters as iaa
import imgaug as ia

Load the Data

In [73]:
df=pd.read_csv('./yolodata/train.csv')

Let us have a look at the counts of the classes to get an idea of the number of images to augment for each class

In [12]:
## Number of Images for each class
for i in range(11):
    print(i,len(df[df['class']==i*1.0].groupby('image_path')))

0 504
1 68
2 910
3 3062
4 825
5 56
6 1
7 820
8 388
9 905
10 58


In [14]:
## Number of Objects for each class
df['class'].value_counts()

3.0     6856
4.0     2167
2.0     2102
9.0     1814
7.0     1242
0.0      929
8.0      599
10.0      99
1.0       91
5.0       68
6.0        1
Name: class, dtype: int64

In [2]:
BASE_IMAGE_PATH="./yolodata/images/train/"
BASE_LABEL_PATH="./yolodata/labels/train/"
## Create these folders before running
NEW_IMAGE_PATH="./yolodata_aug/images/train/"
NEW_LABEL_PATH="./yolodata_aug/labels/train/"

Empirically decided annotation counts for each class based on number of objects and images for each class

In [3]:
augmentations_cnt={
    '0': 5,
    '1': 20,
    '2': 2,
    '3': 0,
    '4': 1,
    '5': 20,
    '6': 2,
    '7': 5,
    '8': 1,
    '9': 20,
}

In [4]:
files = os.listdir(BASE_IMAGE_PATH)

In [ ]:
# final_cnt=defaultdict(int) uncomment to also calculate final counts of the class distribution in the augmented images
all_images=[]
all_bboxes=[]
img_names=[]
num_aug=-1
for id,file in enumerate(tqdm(files)):
    img=np.array(cv2.imread(BASE_IMAGE_PATH+files[0]))

    label_file=BASE_LABEL_PATH+file.replace('jpg','txt')
    class_contri=defaultdict(int)
    
    bboxes=[]
    with open(label_file,'r') as f:
        for line in f.readlines():
            class_contri[line.split()[0]]+=1
            x_c,y_c,w,h=float(line.split()[1])*640, float(line.split()[2])*640, float(line.split()[3])*640, float(line.split()[4][:-1])*640
            bboxes.append(
                BoundingBox(x1=x_c-(w/2), y1=y_c-(h/2), x2=x_c+(w/2), y2=y_c+(h/2),label=line.split()[0]),
            )
    bboxes=BoundingBoxesOnImage(bboxes,shape=img.shape)
    num_aug=0
    for k,v in class_contri.items():
        if k=='3':
            num_aug=0
            break
        num_aug+=augmentations_cnt[k]/v
    num_aug/=len(class_contri)
    num_aug=int(num_aug)
    ## uncomment to also calculate final counts of the class distribution in the augmented images
    # for k,v in class_contri.items():
    #     final_cnt[k]+=(num_aug+1)*v
    all_images.extend([img]*(num_aug+1))
    img_names.extend([file]*(num_aug+1))
    all_bboxes.extend([bboxes]*(num_aug+1))

Counts after augmentation: 

    {'3': 6856,
    '8': 2065,
    '4': 2589,
    '2': 3170,
    '7': 1577,
    '5': 842,
    '6': 2516,
    '0': 2723,
    '9': 863,
    '1': 789}
              
 Total Count: 23990

 Much More balanced than the earlier class distribution

In [8]:
ia.seed(3407)

Defining the augmentations

In [9]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

In [40]:
seq = iaa.Sequential(
    [
        sometimes(iaa.GaussianBlur((0, 3.0))),
        sometimes(iaa.AverageBlur(k=(2, 7))),
        sometimes(iaa.MedianBlur(k=(3, 11))),
        iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
        iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
        iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),
        iaa.LinearContrast((0.5, 2.0), per_channel=0.5),
        sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
        sometimes(
                    iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                ),
    ],random_order=True
)

In [49]:
images_aug,bboxes_aug = seq(images=all_images,bounding_boxes=all_bboxes)

Augmenting the images and saving it in the new folder

In [76]:
for id,img_name in enumerate(img_names):
    label_file=img_name.replace('jpg','txt')
    
    final_annots=[]
    bboxes_aug[id]=bboxes_aug[id].remove_out_of_image().clip_out_of_image()
    for i,bbox in enumerate(bboxes_aug[id]):
        annot_text=str(bbox.label)+" "
        x_1,y_1,x_2,y_2=bbox.x1/640, bbox.y1/640, bbox.x2/640, bbox.y2/640
        annot_text+=str((x_1+x_2)/2)+" "+str((y_1+y_2)/2)+" "+str(x_2-x_1)+" "+str(y_2-y_1)+"\n"
        final_annots.append(annot_text)
    
    with open(NEW_LABEL_PATH+label_file.split('.')[0]+"_"+str(id)+".txt","w") as f:
        f.writelines(final_annots)

    cv2.imwrite(NEW_IMAGE_PATH+img_name.split('.')[0]+"_"+str(id)+".jpg",images_aug[id])